<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

Successfully built ta


  Attempting uninstall: protobuf


    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-09 14:23:20
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16


-------------------
Initial Investment:  1.23 C
CY Investment:  1.50 C
Reserve:  57.44 K
Current:  1.41 C
-------------------
Today PnL: 48.08 K (0.34%)
Current PnL: -20.64 L (-13.75%)
CY Booked + Current PnL: -9.20 L (-6.13%)
-------------------
Total profit:  1.63 L
Total loss:  -22.27 L
-------------------
Total Booked + Current PnL: 17.78 L (14.44%)
Total Booked PnL: 38.42 L (31.2%)
Curr Year Booked PnL: 11.44 L (8.12%)
Prev Year Booked PnL: 26.98 L (21.91%)
Est FTT:  2.29 C
Est FTT PnL: 88.14 L (62.53%)
Deployed:  1.23 C
Current:  1.41 C
CAGR/XIRR %: 8.14%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-39.52,72.0,H-MC,14.63,121253.0,20749.0,8354.0,2.31,20.65,6.89,28.96,79.0,2.48,0.86,61.62,MH,ATH,METALS
77,TTKPRESTIG,770.00,93.81,40.0,M-SC,1.00,83486.0,-17291.0,17382.0,0.31,-17.16,20.82,0.09,245.0,-0.99,0.59,9.68,OX40N,NTT,DURABLES
14,BLUESTARCO,2080.00,9.17,47.0,X-MC,3.04,188560.0,23890.0,19441.0,-0.36,14.51,10.31,26.31,89.0,1.23,1.34,23.65,X40N,NTT,AC
19,CIPLA,1662.74,-22.18,45.0,X-LC,3.67,207212.0,2712.0,20576.0,1.20,1.33,9.93,11.39,10.0,0.13,1.48,10.40,X40N,ATH,PHARMA
84,VOLTAS,1530.00,-1.22,53.0,X-MC,1.17,207345.0,15603.0,22165.0,1.10,8.14,10.69,19.69,99.0,0.70,1.48,15.49,XY25,NTT,AC


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,RAJOOENG,143.10,-39.71,47.0,H-SC,16.35,91360.0,-11140.0,51737.0,5.13,-10.87,56.63,39.61,114.0,-0.22,0.65,11.58,AR,ATH,MISC
67,SONACOMS,1006.00,-40.13,50.0,M-SC,13.71,74462.0,-26697.0,101589.0,5.00,-26.39,136.43,74.03,202.0,-0.26,0.53,5.00,AR,BTT,AUTO
21,COFFEEDAY,80.00,-43.66,51.0,L-SC,29.49,81984.0,-31565.0,71613.0,4.99,-27.80,87.35,35.27,268.0,-0.44,0.58,100.66,XR,NTT,HOTELS
32,HINDZINC,730.22,35.55,73.0,M-LC,7.35,221198.0,16122.0,96442.0,3.67,7.86,43.60,54.89,52.0,0.17,1.58,33.85,X5K,ATH,METALS
4,ANGELONE,3033.00,5.61,52.0,X-SC,11.24,186517.0,-4489.0,62185.0,2.57,-2.35,33.34,30.21,157.0,-0.07,1.33,18.04,X40N,NTT,FINANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BATAINDIA,2096.0,-35.01,43.0,X-SC,4.41,97276.0,-31394.0,76692.0,-2.41,-24.40,78.84,35.20,219.0,-0.41,0.69,11.65,X40,NTT,FOOTWEAR
73,TATAMOTORS,1065.0,-21.50,44.0,X-LC,2.10,247403.0,-29120.0,139189.0,-2.36,-10.53,56.26,39.81,54.0,-0.21,1.76,18.57,XY24,NTT,AUTO
18,CERA,9475.0,-23.87,36.0,H-SC,1.68,138770.0,-37133.0,79154.0,-2.32,-21.11,57.04,23.89,149.0,-0.47,0.99,19.85,OX40N,NTT,CERAMICS
85,WHIRLPOOL,2270.0,-49.50,23.0,M-SC,2.13,85425.0,-6074.0,84827.0,-2.18,-6.64,99.30,86.07,223.0,-0.07,0.61,24.05,XR,NTT,DURABLES
7,ATULAUTO,844.0,3754.05,56.0,M-SC,6.28,173162.0,-23970.0,113802.0,-2.02,-12.16,65.72,45.57,236.0,-0.21,1.23,22.77,XY24,NTT,AUTO


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,106.53,61.0,H-SC,13.54,130275.0,-12204.0,31722.0,1.15,-8.57,24.35,13.70,163.0,-0.38,0.93,53.76,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,93.81,40.0,M-SC,1.00,83486.0,-17291.0,17382.0,0.31,-17.16,20.82,0.09,245.0,-0.99,0.59,9.68,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.97,47.0,H-SC,1.97,220536.0,-49131.0,85458.0,0.18,-18.22,38.75,13.47,138.0,-0.57,1.57,12.14,XY24,NTT,PAINTS
72,TATAELXSI,9161.0,-22.03,54.0,H-MC,6.51,105478.0,-26357.0,68582.0,1.68,-19.99,65.02,32.03,98.0,-0.38,0.75,18.14,OX40N,NTT,IT
18,CERA,9475.0,-23.87,36.0,H-SC,1.68,138770.0,-37133.0,79154.0,-2.32,-21.11,57.04,23.89,149.0,-0.47,0.99,19.85,OX40N,NTT,CERAMICS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-39.52,72.0,H-MC,14.63,121253.0,20749.0,8354.0,2.31,20.65,6.89,28.96,79.0,2.48,0.86,61.62,MH,ATH,METALS


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,46.75,57.0,M-MC,12.58,231523.0,6561.0,161094.0,2.07,2.92,69.58,74.53,192.0,0.04,1.65,36.11,XY24,BTT,STEEL
32,HINDZINC,730.22,35.55,73.0,M-LC,7.35,221198.0,16122.0,96442.0,3.67,7.86,43.60,54.89,52.0,0.17,1.58,33.85,X5K,ATH,METALS
79,UNIONBANK,163.00,-13.18,55.0,M-LC,8.99,157293.0,16453.0,28045.0,1.71,11.68,17.83,31.59,66.0,0.59,1.12,41.26,XY24,NTT,BANKS
86,WIPRO,420.00,-13.34,51.0,M-LC,5.66,152520.0,1575.0,107877.0,0.71,1.04,70.73,72.51,53.0,0.01,1.09,7.13,XR,NTT,IT
11,BANKINDIA,190.00,-25.47,66.0,H-MC,11.51,193342.0,13534.0,98875.0,0.70,7.53,51.14,62.52,88.0,0.14,1.38,44.18,XR,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-39.52,72.0,H-MC,14.63,121253.0,20749.0,8354.0,2.31,20.65,6.89,28.96,79.0,2.48,0.86,61.62,MH,ATH,METALS
32,HINDZINC,730.22,35.55,73.0,M-LC,7.35,221198.0,16122.0,96442.0,3.67,7.86,43.60,54.89,52.0,0.17,1.58,33.85,X5K,ATH,METALS
11,BANKINDIA,190.00,-25.47,66.0,H-MC,11.51,193342.0,13534.0,98875.0,0.70,7.53,51.14,62.52,88.0,0.14,1.38,44.18,XR,NTT,BANKS
37,INDIAMART,4810.62,-54.27,45.0,H-SC,8.95,127894.0,4558.0,127063.0,-1.01,3.70,99.35,106.72,119.0,0.04,0.91,27.94,AR,ATH,MISC
86,WIPRO,420.00,-13.34,51.0,M-LC,5.66,152520.0,1575.0,107877.0,0.71,1.04,70.73,72.51,53.0,0.01,1.09,7.13,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-47.66,25.0,X-SC,37.82,52067.0,-12939.0,163865.0,-0.81,-19.90,314.72,232.18,198.0,-0.08,0.37,0.00,XY24,NTT,MISC
23,DABUR,735.00,-13.02,31.0,X-MC,3.91,235225.0,-12843.0,121258.0,1.01,-5.18,51.55,43.70,102.0,-0.11,1.68,8.52,XY24,BTT,FMCG
5,ASIANPAINT,3465.66,-18.68,32.0,X-LC,5.63,205410.0,-46358.0,99562.0,0.24,-18.41,48.47,21.13,27.0,-0.47,1.46,10.49,X40,ATH,PAINTS
29,HAVELLS,2069.16,-6.46,34.0,X-MC,0.56,231941.0,-15790.0,90851.0,-1.11,-6.37,39.17,30.30,92.0,-0.17,1.65,6.07,X40,ATH,ELECTRICAL
22,COLPAL,3767.14,-5.42,35.0,X-MC,7.08,220980.0,-42385.0,155725.0,-0.35,-16.09,70.47,43.04,84.0,-0.27,1.57,2.59,XY25,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
29,HAVELLS,2069.16,-6.46,34.0,X-MC,0.56,231941.0,-15790.0,90851.0,-1.11,-6.37,39.17,30.30,92.0,-0.17,1.65,6.07,X40,ATH,ELECTRICAL
84,VOLTAS,1530.00,-1.22,53.0,X-MC,1.17,207345.0,15603.0,22165.0,1.10,8.14,10.69,19.69,99.0,0.70,1.48,15.49,XY25,NTT,AC
34,ICICIGI,2260.25,-19.18,47.0,X-MC,1.81,139016.0,2883.0,28248.0,0.03,2.12,20.32,22.86,91.0,0.10,0.99,15.00,X40,ATH,INSURANCE
65,SIEMENS,4671.50,-1.11,49.0,H-LC,1.95,159030.0,-27065.0,74537.0,-1.13,-14.54,46.87,25.51,15.0,-0.36,1.13,17.19,AR,ATH,ELECTRICAL
52,PGHH,17907.65,-26.84,60.0,X-MC,2.06,211800.0,10980.0,56805.0,-1.18,5.47,26.82,33.76,80.0,0.19,1.51,10.95,X40,ATH,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-47.66,25.0,X-SC,37.82,52067.0,-12939.0,163865.0,-0.81,-19.90,314.72,232.18,198.0,-0.08,0.37,0.00,XY24,NTT,MISC
83,VBL,671.64,-21.55,37.0,X-LC,10.31,282347.0,-33495.0,146171.0,1.77,-10.60,51.77,35.67,5.0,-0.23,2.01,1.78,X40N,ATH,FMCG
22,COLPAL,3767.14,-5.42,35.0,X-MC,7.08,220980.0,-42385.0,155725.0,-0.35,-16.09,70.47,43.04,84.0,-0.27,1.57,2.59,XY25,ATH,FMCG
42,ITC,452.00,-39.61,40.0,X-LC,2.27,195404.0,-4734.0,25617.0,-0.04,-2.37,13.11,10.44,4.0,-0.18,1.39,3.07,X40,NTT,FMCG
9,BAJAJHFL,181.50,-14.27,39.0,X-MC,4.87,180630.0,-19876.0,117030.0,-0.37,-9.91,64.79,48.45,90.0,-0.17,1.29,4.16,X40N,ATH,FINANCE


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-47.66,25.0,X-SC,37.82,52067.0,-12939.0,163865.0,-0.81,-19.90,314.72,232.18,198.0,-0.08,0.37,0.00,XY24,NTT,MISC
56,RELAXO,1176.00,-39.49,40.0,X-SC,2.17,82476.0,-62684.0,138609.0,-0.89,-43.18,168.06,52.31,136.0,-0.45,0.59,10.40,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-35.01,43.0,X-SC,4.41,97276.0,-31394.0,76692.0,-2.41,-24.40,78.84,35.20,219.0,-0.41,0.69,11.65,X40,NTT,FOOTWEAR
17,CAMS,5211.76,-6.05,47.0,X-SC,1.65,106935.0,4931.0,38999.0,1.03,4.83,36.47,43.06,122.0,0.13,0.76,22.78,X40N,ATH,MISC
33,HONAUT,58357.33,-23.12,44.0,X-SC,2.16,108330.0,-19608.0,66742.0,-0.18,-15.33,61.61,36.84,143.0,-0.29,0.77,10.90,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4406.34,-28.36,52.0,X-LC,12.17,285666.0,-60290.0,128521.0,0.39,-17.43,44.99,19.73,1.0,-0.47,2.03,5.21,X40,ATH,IT
40,INFY,2275.00,-17.80,55.0,X-LC,7.91,319798.0,6910.0,164792.0,0.43,2.21,51.53,54.87,3.0,0.04,2.28,9.00,X40,BTT,IT
42,ITC,452.00,-39.61,40.0,X-LC,2.27,195404.0,-4734.0,25617.0,-0.04,-2.37,13.11,10.44,4.0,-0.18,1.39,3.07,X40,NTT,FMCG
83,VBL,671.64,-21.55,37.0,X-LC,10.31,282347.0,-33495.0,146171.0,1.77,-10.60,51.77,35.67,5.0,-0.23,2.01,1.78,X40N,ATH,FMCG
1,ABB,7934.00,-41.23,39.0,H-LC,7.53,245472.0,-16147.0,135353.0,-0.28,-6.17,55.14,45.57,7.0,-0.12,1.75,4.45,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-43.66,51.0,L-SC,29.49,81984.0,-31565.0,71613.0,4.99,-27.80,87.35,35.27,268.0,-0.44,0.58,100.66,XR,NTT,HOTELS
49,MASFIN,398.61,-20.96,39.0,H-SC,10.58,89880.0,-8100.0,29705.0,-1.45,-8.27,33.05,22.05,152.0,-0.27,0.64,30.65,XR,ATH,FINANCE
50,NATIONALUM,244.55,-39.52,72.0,H-MC,14.63,121253.0,20749.0,8354.0,2.31,20.65,6.89,28.96,79.0,2.48,0.86,61.62,MH,ATH,METALS
78,UJJIVANSFB,60.00,106.53,61.0,H-SC,13.54,130275.0,-12204.0,31722.0,1.15,-8.57,24.35,13.70,163.0,-0.38,0.93,53.76,OX40N,NTT,BANKS
61,SAMMAANCAP,326.00,-184.09,65.0,M-SC,14.38,144963.0,-4257.0,148442.0,1.25,-2.85,102.40,96.62,208.0,-0.03,1.03,56.88,XY25,NTT,FINANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAMMAANCAP,326.00,-184.09,65.0,M-SC,14.38,144963.0,-4257.0,148442.0,1.25,-2.85,102.40,96.62,208.0,-0.03,1.03,56.88,XY25,NTT,FINANCE
32,HINDZINC,730.22,35.55,73.0,M-LC,7.35,221198.0,16122.0,96442.0,3.67,7.86,43.60,54.89,52.0,0.17,1.58,33.85,X5K,ATH,METALS
69,SURYODAY,240.00,64.47,64.0,H-SC,8.10,150929.0,-28142.0,93878.0,2.33,-15.72,62.20,36.71,135.0,-0.30,1.08,49.75,XR,NTT,BANKS
26,GREENPANEL,537.00,237.21,62.0,M-SC,4.94,150450.0,-32628.0,111604.0,0.34,-17.82,74.18,43.14,230.0,-0.29,1.07,41.52,XY24,NTT,MISC
50,NATIONALUM,244.55,-39.52,72.0,H-MC,14.63,121253.0,20749.0,8354.0,2.31,20.65,6.89,28.96,79.0,2.48,0.86,61.62,MH,ATH,METALS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.15
1,25,43.47
2,50,75.17


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.94
LC    32.30
MC    23.74
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.94
X40      14.62
X40N     11.21
XR       10.93
XY25     10.39
AR        8.17
OX40N     7.67
X200      1.73
MH        1.62
X5K       1.58
SR        1.12
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    25.49
X-LC    22.89
X-MC    17.16
M-SC    12.51
H-MC     4.63
H-LC     4.61
X-SC     4.51
M-LC     3.79
M-MC     1.65
L-SC     1.43
L-LC     1.01
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.75,-6.36,41.41
IT,12.88,-16.35,79.37
FINANCE,10.48,-14.73,70.14
MISC,6.93,-18.09,83.51
BANKS,6.36,-11.85,70.69
PAINTS,5.50,-19.61,37.44
ELECTRICAL,5.30,-10.55,49.34
INSURANCE,3.80,-3.58,42.30
AUTO,3.52,-16.12,71.63


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3227987.0,22
XR,1374932.0,14
AR,1184121.0,9
X40,785907.0,10
X40N,662102.0,9
OX40N,568821.0,10
XY25,479533.0,7
SR,265116.0,2
X5K,96442.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3424719.0,25
M-SC,1281962.0,15
X-MC,1126190.0,12
X-LC,1027542.0,13
X-SC,547092.0,6
H-MC,358639.0,4
H-LC,292935.0,3
L-SC,259243.0,3
M-LC,232364.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1206072.0      6
           AR         806881.0      5
           XR         804053.0      7
M-SC       XY24       685606.0      6
X-MC       XY24       591960.0      4
X-LC       X40        489346.0      5
X-SC       X40N       306535.0      4
H-SC       SR         265116.0      2
           OX40N      264591.0      4
X-MC       X40        219869.0      4
X-LC       X40N       219096.0      3
H-LC       AR         209890.0      2
X-LC       XY24       208517.0      2
H-MC       XY24       182828.0      1
X-MC       XY25       177890.0      2
L-SC       XR         170154.0      2
M-SC       AR         167350.0      2
X-SC       XY24       163865.0      1
M-MC       XY24       161094.0      1
M-SC       XY25       148442.0      1
           OX40N      146559.0      4
X-MC       X40N       136471.0      2
M-SC       XR         134005.0      2
X-LC       XY25       110583.0      3
M-LC       XR         107877.0      1
H-MC       XR          98875.0      1
M-LC       X5K         96442.0      1
L-SC       OX40N       89089.0      1
H-LC       X200        83045.0      1
H-SC       MH          78006.0      1
X-SC       X40         76692.0      1
H-MC       OX40N       68582.0      1
L-MC       XR          59968.0      1
L-LC       XY25        42618.0      1
M-LC       XY24        28045.0      1
H-MC       MH           8354.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 31.0 seconds
